In [1]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [2]:
BAND = "r"
NON_EXTENDED_ONLY = True

In [3]:
N_SRC = 10
BATCH_SIZE = 32
N_LCS = 400_000

In [4]:
DP1_ROOT = "../../data/dp1"
LSDB_WORKERS = 8

In [ ]:
from uncle_val.pipelines import run_dp1_linear_flux_err

run_dp1_linear_flux_err(
    dp1_root=DP1_ROOT,
    band=BAND,
    non_extended_only=NON_EXTENDED_ONLY,
    n_workers=LSDB_WORKERS,
    n_src=N_SRC,
    n_lcs=N_LCS,
    train_batch_size=BATCH_SIZE,
    start_tfboard=False,
    output_root="./runs",
)

/Users/hombit/.virtualenvs/uncle-val/lib/python3.13/site-packages/tensorboard/data/server_ingester.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Dask Dashboard Link: http://127.0.0.1:8787/status
